In [1]:
import numpy as np
import pandas as pd

In [2]:
def ddataset_balance(X_temp, y_temp):
  smote = SMOTE()
  X_temp, y_temp= smote.fit_resample(X_temp, y_temp)
  return pd.concat([pd.DataFrame(X_temp), pd.DataFrame(y_temp)], axis=1)

def Xy_balance(X_temp, y_temp):
  smote = SMOTE()
  return smote.fit_resample(X_temp, y_temp)

def Search_Null(dataset):
  dic={}
  for col in list(dataset.columns):
    rows = []
    flage = 0
    for row in range(dataset.shape[0]):
      if str(dataset[col][row]) == "nan":
        rows.append(row)
        flage = 1
    if flage ==1:
      dic[col] = rows
  
  return dic


def models_check_box(models):
  from IPython.display import display
  new_keys=[]
  for i in models:
    i=widgets.Checkbox(
      value=False,
      description=str(i),
      disabled=False,
      indent=False
      )
    display(i)
    new_keys.append(i)
  return new_keys


def divide_dataset(dataset,percentage,random_state = 0):

  rows,cols = dataset.shape

  if random_state == 0:
    return dataset.iloc[:int(rows*(percentage)),:]
  else:
    import random
    l_r = random.sample(range(0, rows), int(rows*(percentage)))
    return dataset.iloc[l_r,:]

def StandardScaleData(data):
  scaler = StandardScaler()
  scaler.fit(data)
  return  scaler.transform(data)


def MinMaxScaleData(data):
  scaler = MinMaxScaler()
  scaler.fit(data)
  return scaler.transform(data)

In [3]:
dataset = pd.read_csv("https://raw.githubusercontent.com/RafsanJany-44/Machine-School/main/datasets/FiveStageSleep_Small_dataset.csv")

target = "Sleep Stage"
X =  dataset.loc[:,dataset.columns != target]   # removing Sleep Stage
X =  X.loc[:,X.columns != "Subject"]            # removing Status
X =  X.loc[:,X.columns != "Epoch"]              # removing Epoch
y = dataset[target]


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [5]:
dataset[target].value_counts()

N2    1884
W      926
N3     827
R      793
N1     570
Name: Sleep Stage, dtype: int64

In [6]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf_default = RandomForestClassifier(random_state=0,n_jobs=-1)
rf_default.fit(X_train, y_train)
y_pred=rf_default.predict(X_test)



from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

[[ 56  14   0   9  31]
 [ 14 342  12   8   8]
 [  0  23 152   0   1]
 [  9   6   1 131   6]
 [ 13   3   0   2 159]]
              precision    recall  f1-score   support

          N1       0.61      0.51      0.55       110
          N2       0.88      0.89      0.89       384
          N3       0.92      0.86      0.89       176
           R       0.87      0.86      0.86       153
           W       0.78      0.90      0.83       177

    accuracy                           0.84      1000
   macro avg       0.81      0.80      0.81      1000
weighted avg       0.84      0.84      0.84      1000

Accurecy:  0.84
CPU times: user 2.02 s, sys: 41.3 ms, total: 2.06 s
Wall time: 1.39 s


In [7]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 12.2 MB/s eta 0:00:00


In [8]:
%%time
from catboost import CatBoostClassifier

model = CatBoostClassifier(loss_function='MultiClass')

model.fit(X_train, y_train,
          verbose=False)



y_pred=model.predict(X_test)



from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))


[[ 63  13   0   8  26]
 [ 15 345  13   5   6]
 [  0  20 155   0   1]
 [ 10   5   0 133   5]
 [ 12   1   0   2 162]]
              precision    recall  f1-score   support

          N1       0.63      0.57      0.60       110
          N2       0.90      0.90      0.90       384
          N3       0.92      0.88      0.90       176
           R       0.90      0.87      0.88       153
           W       0.81      0.92      0.86       177

    accuracy                           0.86      1000
   macro avg       0.83      0.83      0.83      1000
weighted avg       0.86      0.86      0.86      1000

Accurecy:  0.858
CPU times: user 1min 17s, sys: 677 ms, total: 1min 18s
Wall time: 42.7 s


In [9]:
%%time
from catboost import CatBoostClassifier


model = CatBoostClassifier(task_type="GPU",loss_function='MultiClass')

model.fit(X_train, y_train,
          verbose=False)



y_pred=model.predict(X_test)



from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))


[[ 64  12   0   8  26]
 [ 14 350  10   4   6]
 [  0  21 154   0   1]
 [ 11   4   0 132   6]
 [ 13   0   0   1 163]]
              precision    recall  f1-score   support

          N1       0.63      0.58      0.60       110
          N2       0.90      0.91      0.91       384
          N3       0.94      0.88      0.91       176
           R       0.91      0.86      0.89       153
           W       0.81      0.92      0.86       177

    accuracy                           0.86      1000
   macro avg       0.84      0.83      0.83      1000
weighted avg       0.86      0.86      0.86      1000

Accurecy:  0.863
CPU times: user 9.08 s, sys: 5.54 s, total: 14.6 s
Wall time: 8.87 s


In [10]:
import sys
import numba 
import numpy

print("Python version:", sys.version)
print("Numba version:", numba.__version__)
print("Numpy version:", numpy.__version__)

Python version: 3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0]
Numba version: 0.56.4
Numpy version: 1.21.6


In [11]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [12]:
from numba import cuda
import numpy as np

In [13]:
@cuda.jit
def cudakernel0(array):
    for i in range(array.size):
        array[i] += 0.5

In [14]:
array = np.array([0, 1], np.float32)
print('Initial array:', array)

print('Kernel launch: cudakernel0[1, 1](array)')
cudakernel0[1, 1](array)

print('Updated array:',array)

Initial array: [0. 1.]
Kernel launch: cudakernel0[1, 1](array)


/usr/local/lib/python3.8/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Updated array: [0.5 1.5]


/usr/local/lib/python3.8/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [15]:
!pip install scikit-cuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.8/114.8 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 KB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp38-cp38-linux_x86_64.whl size=661775 sha256=c3ab29f1599621ec7d7d149a68655dd7719942f2f0430b8f43ead69df6afabb5
  Stored in directory: /root/.cache/pip/wheels/7b/41/0d/7cecb04af969d283ebe4a69579a8b2baec0d010a1ac4159f7e
  Created wheel for pytools: filename=pytools-2022.1.14-py2.py3-none-any.whl size=69870 sha256=ca43e2983633f0e2fb83e0d0b36ea43685e12